### 데이터의 다양한 전처리 방법 확인

### 학습 내용
 * 정규화, 표준화에 대해 실습을 통해 알아봅니다.
 * 로그 변환에 대해 실습을 통해 알아봅니다.
 * Box-Cox 변환에 대해 실습을 통해 알아봅니다.
 * Yeo-Johnson변환에 대해 실습을 통해 알아봅니다.
 * clipping, binning, 순위, RankGauss로 변환하는 것에 대해 실습을 통해 알아봅니다.

### 데이터 준비

In [9]:
import numpy as np
import pandas as pd

# train_x는 학습 데이터(X), train_y는 Target(예측할 변수), test_x는 테스트 데이터
# pandas의 DataFrame, Series의 자료형 사용(numpy의 array로 값을 저장하기도 함.)
train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

# 학습 데이터와 테스트 데이터의 원래 상태를 복제해 두기
train_x_saved = train_x.copy()
test_x_saved = test_x.copy()

# 학습 데이터와 테스트 데이터를 반환하는 함수
def load_data():
    train_x, test_x = train_x_saved.copy(), test_x_saved.copy()
    return train_x, test_x

In [10]:
# 대상 특징(피처) 지정 - 수치 변수를 목록에 저장
num_cols = ['age', 'height', 'weight', 'amount',
            'medical_info_a1', 'medical_info_a2', 'medical_info_a3', 'medical_info_b1']



### 데이터 전처리 - 표준화
  * 평균을 0으로, 표준편차를 1로 변경
  * 기준이 되는 값은 학습 데이터 셋(train_x)

In [20]:
# 데이터 읽어오기
train_x, test_x = load_data()

from sklearn.preprocessing import StandardScaler

# 학습 데이터를 기반으로 복수 열의 표준화를 정의(평균 0, 표준편차 1)
scaler = StandardScaler()
scaler.fit(train_x[num_cols])

# 표준화를 수행한 후 각 열을 치환
train_x[num_cols] = scaler.transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])

# 결과적으로 평균은 0, 표준편차는 1로 매우 가깝게 만들어짐.
print( train_x[num_cols].mean(), train_x[num_cols].std() )
print( test_x[num_cols].mean(), test_x[num_cols].std() )

age                6.221690e-17
height            -2.822720e-15
weight            -1.951661e-16
amount            -2.140954e-16
medical_info_a1   -4.767020e-17
medical_info_a2   -3.677059e-17
medical_info_a3    1.968870e-16
medical_info_b1    2.520206e-16
dtype: float64 age                1.00005
height             1.00005
weight             1.00005
amount             1.00005
medical_info_a1    1.00005
medical_info_a2    1.00005
medical_info_a3    1.00005
medical_info_b1    1.00005
dtype: float64
age                0.002597
height             0.005582
weight            -0.017946
amount            -0.013080
medical_info_a1    0.005988
medical_info_a2    0.008365
medical_info_a3    0.002117
medical_info_b1    0.025981
dtype: float64 age                1.003940
height             0.989233
weight             1.004621
amount             1.004529
medical_info_a1    1.003076
medical_info_a2    0.997363
medical_info_a3    0.992721
medical_info_b1    0.990133
dtype: float64


### 데이터 전처리 - 표준화
  * 평균을 0으로, 표준편차를 1로 변경
  * 기준이 되는 값은 학습/테스트 결합 데이터 셋(train_x, test_x)

In [19]:
# -----------------------------------
# 데이터 읽어오기
train_x, test_x = load_data()
# -----------------------------------
from sklearn.preprocessing import StandardScaler

# 학습 데이터와 테스트 데이터를 결합한 결과를 기반으로 복수 열의 표준화를 정의
scaler = StandardScaler()
scaler.fit(pd.concat([train_x[num_cols], test_x[num_cols]]))

# 표준화 변환 후 데이터로 각 열을 치환
train_x[num_cols] = scaler.transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])

# 결과적으로 평균은 0, 표준편차는 1로 매우 가깝게 만들어짐.
print( train_x[num_cols].mean(), train_x[num_cols].std() )
print( test_x[num_cols].mean(), test_x[num_cols].std() )

age               -0.001296
height            -0.002806
weight             0.008952
amount             0.006525
medical_info_a1   -0.002989
medical_info_a2   -0.004188
medical_info_a3   -0.001062
medical_info_b1   -0.013054
dtype: float64 age                0.998106
height             1.005469
weight             0.997727
amount             0.997792
medical_info_a1    0.998534
medical_info_a2    1.001385
medical_info_a3    1.003721
medical_info_b1    1.004935
dtype: float64
age                0.001296
height             0.002806
weight            -0.008952
amount            -0.006525
medical_info_a1    0.002989
medical_info_a2    0.004188
medical_info_a3    0.001062
medical_info_b1    0.013054
dtype: float64 age                1.001988
height             0.994593
weight             1.002288
amount             1.002261
medical_info_a1    1.001555
medical_info_a2    0.998695
medical_info_a3    0.996365
medical_info_b1    0.994969
dtype: float64


### 데이터 전처리 - 표준화 (나쁜 예제)
  * 평균을 0으로, 표준편차를 1로 변경
  * 기준이 되는 값은 학습, 테스트 데이터 각각 정함.  나쁜 예

In [18]:
# -----------------------------------
# 데이터 읽어오기
train_x, test_x = load_data()
# -----------------------------------
from sklearn.preprocessing import StandardScaler

# 학습 데이터와 테스트 데이터를 각각 표준화(나쁜 예)
scaler_train = StandardScaler()
scaler_train.fit(train_x[num_cols])
train_x[num_cols] = scaler_train.transform(train_x[num_cols])

scaler_test = StandardScaler()
scaler_test.fit(test_x[num_cols])
test_x[num_cols] = scaler_test.transform(test_x[num_cols])

# 결과적으로 평균은 0, 표준편차는 1로 매우 가깝게 만들어짐.
print( train_x[num_cols].mean(), train_x[num_cols].std() )
print( test_x[num_cols].mean(), test_x[num_cols].std() )

age                6.221690e-17
height            -2.822720e-15
weight            -1.951661e-16
amount            -2.140954e-16
medical_info_a1   -4.767020e-17
medical_info_a2   -3.677059e-17
medical_info_a3    1.968870e-16
medical_info_b1    2.520206e-16
dtype: float64 age                1.00005
height             1.00005
weight             1.00005
amount             1.00005
medical_info_a1    1.00005
medical_info_a2    1.00005
medical_info_a3    1.00005
medical_info_b1    1.00005
dtype: float64
age                5.845689e-17
height            -2.455292e-15
weight             2.793654e-16
amount            -3.348322e-16
medical_info_a1   -1.731948e-18
medical_info_a2   -9.181544e-17
medical_info_a3    5.728751e-18
medical_info_b1    4.193951e-16
dtype: float64 age                1.00005
height             1.00005
weight             1.00005
amount             1.00005
medical_info_a1    1.00005
medical_info_a2    1.00005
medical_info_a3    1.00005
medical_info_b1    1.00005
dtype: floa

### 데이터 전처리 - 정규화(Min-Max 스케일링)
  * 값을 0~1 사이로 변환

In [22]:
# 데이터 읽어오기
train_x, test_x = load_data()
# -----------------------------------
from sklearn.preprocessing import MinMaxScaler

# 학습 데이터를 기반으로 여러 열의 최소-최대 스케일링 정의
scaler = MinMaxScaler()
scaler.fit(train_x[num_cols])

# 정규화(0~1) 변환 후의 데이터로 각 열을 치환
train_x[num_cols] = scaler.transform(train_x[num_cols])
test_x[num_cols] = scaler.transform(test_x[num_cols])

# 결과적으로 평균은 0, 표준편차는 1로 매우 가깝게 만들어짐.
print( train_x[num_cols].min(), train_x[num_cols].max() )
print( test_x[num_cols].min(), test_x[num_cols].max() )

age                0.0
height             0.0
weight             0.0
amount             0.0
medical_info_a1    0.0
medical_info_a2    0.0
medical_info_a3    0.0
medical_info_b1    0.0
dtype: float64 age                1.0
height             1.0
weight             1.0
amount             1.0
medical_info_a1    1.0
medical_info_a2    1.0
medical_info_a3    1.0
medical_info_b1    1.0
dtype: float64
age                0.000000
height            -0.022861
weight            -0.002340
amount             0.000000
medical_info_a1   -0.067708
medical_info_a2   -0.047919
medical_info_a3    0.000000
medical_info_b1    0.000000
dtype: float64 age                1.000000
height             1.050548
weight             0.927204
amount             1.000000
medical_info_a1    0.963542
medical_info_a2    1.003783
medical_info_a3    1.000000
medical_info_b1    1.000000
dtype: float64


### 데이터 전처리 - 로그 변환

In [25]:
# -----------------------------------
# 로그 변환
# -----------------------------------
x = np.array([1.0, 10.0, 100.0, 1000.0, 10000.0])

# 단순히 값에 로그를 취함
x1 = np.log(x)

# 1을 더한 뒤에 로그를 취함
x2 = np.log1p(x)

# 절댓값의 로그를 취한 후, 원래의 부호를 추가
x3 = np.sign(x) * np.log(np.abs(x))

print("원래 값 : ", x)
print("단순로그 값 x1: ", x1)
print("log1p로그 값 x2: ", x2)
print("x3: ", x3)


원래 값 :  [1.e+00 1.e+01 1.e+02 1.e+03 1.e+04]
단순로그 값 x1:  [0.         2.30258509 4.60517019 6.90775528 9.21034037]
log1p로그 값 x2:  [0.69314718 2.39789527 4.61512052 6.90875478 9.21044037]
x3:  [0.         2.30258509 4.60517019 6.90775528 9.21034037]


### 데이터 전처리 - Box-Cox 변환

In [26]:
# 데이터 읽어오기
train_x, test_x = load_data()
# -----------------------------------

# 양의 정숫값만을 취하는 변수를 변환 대상으로 목록에 저장
# 또한, 결측값을 포함하는 경우는 (~(train_x[c] <= 0.0)).all() 등으로 해야 하므로 주의
pos_cols = [c for c in num_cols if (train_x[c] > 0.0).all() and (test_x[c] > 0.0).all()]

from sklearn.preprocessing import PowerTransformer

# 학습 데이터를 기반으로 복수 열의 박스-칵스 변환 정의
pt = PowerTransformer(method='box-cox')
pt.fit(train_x[pos_cols])

# 변환 후의 데이터로 각 열을 치환
train_x[pos_cols] = pt.transform(train_x[pos_cols])
test_x[pos_cols] = pt.transform(test_x[pos_cols])

### 데이터 전처리 - Yeo-Johnson변환

In [28]:
# 데이터 읽어오기
train_x, test_x = load_data()
# -----------------------------------

from sklearn.preprocessing import PowerTransformer

# 학습 데이터를 기반으로 복수 열의 여-존슨 변환 정의
pt = PowerTransformer(method='yeo-johnson')
pt.fit(train_x[num_cols])

# 변환 후의 데이터로 각 열을 치환
train_x[num_cols] = pt.transform(train_x[num_cols])
test_x[num_cols] = pt.transform(test_x[num_cols])

### 데이터 전처리 - Clipping

In [29]:
# -----------------------------------
# clipping
# -----------------------------------
# 데이터 읽어오기
train_x, test_x = load_data()
# -----------------------------------
# 열마다 학습 데이터의 1%, 99% 지점을 확인
p01 = train_x[num_cols].quantile(0.01)
p99 = train_x[num_cols].quantile(0.99)

# 1％점 이하의 값은 1%점으로, 99%점 이상의 값은 99%점으로 클리핑
train_x[num_cols] = train_x[num_cols].clip(p01, p99, axis=1)
test_x[num_cols] = test_x[num_cols].clip(p01, p99, axis=1)

C:\Users\daniel_wj\AppData\Local\Temp\ipykernel_18644\3007283885.py:12: FutureWarning: Downcasting integer-dtype results in .where is deprecated and will change in a future version. To retain the old behavior, explicitly cast the results to the desired dtype.
  train_x[num_cols] = train_x[num_cols].clip(p01, p99, axis=1)
C:\Users\daniel_wj\AppData\Local\Temp\ipykernel_18644\3007283885.py:13: FutureWarning: Downcasting integer-dtype results in .where is deprecated and will change in a future version. To retain the old behavior, explicitly cast the results to the desired dtype.
  test_x[num_cols] = test_x[num_cols].clip(p01, p99, axis=1)


### 데이터 전처리 - binning

In [31]:
x = [1, 7, 5, 4, 6, 3]

# 팬더스 라이브러리의 cut 함수로 구간분할 수행

# bin의 수를 지정할 경우
binned = pd.cut(x, 3, labels=False)
print(binned)
# [0 2 1 1 2 0] - 변환된 값은 세 구간(0, 1, 2)를 만들고 원본 x의 값이 어디에 해당되는지 나타냄

# bin의 범위를 지정할 경우(3.0 이하, 3.0보다 크고 5.0보다 이하, 5.0보다 큼)
bin_edges = [-float('inf'), 3.0, 5.0, float('inf')]
binned = pd.cut(x, bin_edges, labels=False)
print(binned)
# [0 2 1 1 2 0] - 변환된 값은 세 구간을 만들고 원본 x의 값이 어디에 해당되는지 나타냄

[0 2 1 1 2 0]
[0 2 1 1 2 0]


### 데이터 전처리 - 순위로 변환

In [33]:
x = [10, 20, 30, 0, 40, 40]

# 팬더스의 rank 함수로 순위 변환
rank = pd.Series(x).rank()
print(rank.values)
# 시작이 1, 같은 순위가 있을 경우에는 평균 순위가 됨
# [2. 3. 4. 1. 5.5 5.5]

# 넘파이의 argsort 함수를 2회 적용하는 방법으로 순위 변환
order = np.argsort(x)
rank = np.argsort(order)
print(rank)
# 넘파이의 argsort 함수를 2회 적용하는 방법으로 순위 변환
# [1 2 3 0 4 5]

[2.  3.  4.  1.  5.5 5.5]
[1 2 3 0 4 5]


### 데이터 전처리 - RankGauss

In [34]:
# 데이터 읽어오기
train_x, test_x = load_data()
# -----------------------------------
from sklearn.preprocessing import QuantileTransformer

# 학습 데이터를 기반으로 복수 열의 RankGauss를 통한 변환 정의
transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution='normal')
transformer.fit(train_x[num_cols])

# 변환 후의 데이터로 각 열을 치환
train_x[num_cols] = transformer.transform(train_x[num_cols])
test_x[num_cols] = transformer.transform(test_x[num_cols])